Artifical Start

In [202]:
using LinearAlgebra
using SparseArrays
using LightGraphs
using SimpleWeightedGraphs
using GraphPlot
using BenchmarkTools
using Random
include("minFlow.jl")

AddToTable!

In [203]:
nodes = 5
E = [(1,2) (1,3) (1,4) (2,5) (3,5) (4,5)]
cost_dict = Dict((1,2)=>1, (1,3)=>1, (1,4)=>1, (2,5)=>1, (3,5)=>1, (4,5)=>1)

## Artifical Start
n = nodes + 1 
b = Int64[5;-1;-1;-1;-2;0] # Flow
c = Int64[1;1;1;1;1;0] # Artifical Cost
x = abs.(b)

6-element Vector{Int64}:
 5
 1
 1
 1
 2
 0

In [204]:
nodes = 5
E = [(1,2) (1,3) (1,5) (2,3) (3,4) (4,2) (4,5) (5,3)]
cost_dict = Dict((1,2)=> 1,(1,3)=> 2, (1,5)=> 1, (2,3)=> 2, (3,4)=> 1, (4,2)=> 1, (4,5)=> 1, (5,3)=> 1)

## Artifical Start
n = nodes + 1 
b = Int64[2;5;1;-4;-4;0] # Flow
c = Int64[1;1;1;1;1;0] # Artifical Cost
x = abs.(b)

6-element Vector{Int64}:
 2
 5
 1
 4
 4
 0

In [205]:
B = ArtificalBase_table(b)
B_edgeList = ArtificalBase_list(b)
B_matrix = BaseMatrix(B_edgeList,n)
NB_edgeList = copy(E)


1×8 Matrix{Tuple{Int64, Int64}}:
 (1, 2)  (1, 3)  (1, 5)  (2, 3)  (3, 4)  (4, 2)  (4, 5)  (5, 3)

In [206]:
w = zeros(n,1)
valid = true
while (sum(c) != 0)
    w = c'/B_matrix    
    max ,enteringEdge = ReducedCost(w,NB_edgeList,1, 0)
    
    println("Entering:  $max , $enteringEdge")
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,n,B)
    println("Entering and Path ($edge_u ,$edge_v)", path)
    Flow_list, valid = Get_flow(path)
    println(Flow_list, valid)
    if valid
        indexes = Get_Indexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        println("Exiting: $exitingEdge")
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,n,B)
        RemoveFromTable!(edge_j,edge_i,n,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        Update_BaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        c[min_index] = 0
    end
    for (i,e) in enumerate(NB_edgeList) 
        if e == enteringEdge
            NB_edgeList[i] = (0,0)
            break
        end
    end
end

Entering:  2.0 , (1, 5)
Entering and Path (1 ,5)[(1, 5), (1, 6), (6, 5)]
[1, -1, -1]true
Exiting: (1, 6)
Entering:  2.0 , (3, 4)
Entering and Path (3 ,4)[(3, 4), (3, 6), (6, 4)]
[1, -1, -1]true
Exiting: (3, 6)
Entering:  2.0 , (2, 3)
Entering and Path (2 ,3)[(2, 3), (2, 6), (6, 4), (3, 4)]
[1, -1, -1, 1]true
Exiting: (6, 4)
Entering:  2.0 , (4, 5)
Entering and Path (4 ,5)[(4, 5), (3, 4), (2, 3), (2, 6), (6, 5)]
[1, 1, 1, -1, -1]true
Exiting: (6, 5)
Entering:  0.0 , (1, 2)
ENDING!!


In [207]:
basicEdge =  Base_list(B_edgeList, nodes)
cost = Set_cost(basicEdge, cost_dict)
NB_edgeList = NonBasic_list(basicEdge, E)
B_matrix = BaseMatrix(basicEdge,nodes)
B = Base_table(basicEdge, nodes)

b = b[1:end-1] # Flow
x = x[1:end-1]

(1, 2)
(1, 3)
(4, 2)
(5, 3)


5-element Vector{Int64}:
 2
 0
 6
 5
 2

In [208]:
w = cost'/B_matrix
while sum((w*B_matrix)' .>= cost) != nodes
    
    max ,enteringEdge = ReducedCost(w,NB_edgeList, 2, cost_dict)
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,nodes,B)
    Flow_list, valid = getFlow(path,nodes)
    if valid
        indexes = Get_Indexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,nodes,B)
        RemoveFromTable!(edge_j,edge_i,nodes,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        updateBaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        cost[min_index] = cost_dict[enteringEdge]
    end
    w = cost'/B_matrix
end

In [209]:
#Check Feasibility
(w*B_matrix)' .>= cost

5×1 BitMatrix:
 1
 1
 1
 1
 1

In [210]:
x

5-element Vector{Int64}:
 2
 0
 6
 5
 2

In [211]:
cost

5×1 Matrix{Float64}:
 1.0
 0.0
 1.0
 2.0
 1.0

In [212]:
x

5-element Vector{Int64}:
 2
 0
 6
 5
 2

In [213]:
#Min Cost
cost'x

1-element Vector{Float64}:
 20.0